# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Проект рекомендация тарифов

**Цель проекта - исследование моделей машиного обучения для задачи классификации.**\
\
**Ход проекта:**
* **Загрузка и знакомство с данными.**
* **Разделение данных три выборки, train, valid, test.**
* **Исследование моделей - DecisionTreeClassifier, RandomForestClassifier.**
* **Проверка моделей на тестовой выборке.**
* **Проверка модели на адекватность с помощью DummyClassifier.**

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier

from sklearn.model_selection import train_test_split

## Откройте и изучите файл

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Столбцы calls и messages могут являться только натуральным числом, потому изменяем тип данных на int.**

In [3]:
data['calls'] = data['calls'].astype(int)
data['minutes'] = data['minutes'].astype(int)

### Промежуточный вывод:

**Данные состоят из 5 столбцов, и 3214 строк.**\
\
**Столбцы:**\
**calls - количество звонков.**\
**minutes - количество минут.**\
**messages - количество сообщений.**\
**mb_used - количество израсходованных мегабайт.**\
**is_ultra - 1 - ультра, 0 - смарт тариф.**\
\
**Столбцы calls и messages изменен тип данных на int.**

## Разбейте данные на выборки

In [4]:
data_train, data_valid, data_test = np.split(data.sample(frac=1, random_state=12345), 
                                                         [int(.6*len(data)), int(.8*len(data))])

In [5]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

**Данные разбиты на три датафрейма:\
train - 60%\
valid - 20%\
test  - 20%\
Процент от общего числа строк.**

## Исследуйте модели

### Модель DecisionTreeClassifier.

In [28]:
best_model = None 
best_result = 0

for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        
print('Accuracy лучшей модели:', best_result)
print('Лучшая модель:', best_model)

Accuracy лучшей модели: 0.7776049766718507
Лучшая модель: DecisionTreeClassifier(max_depth=10, random_state=12345)


**На валидационной выборке модель DecisionTreeClassifier с гиперпараметром глубины 10 предсказывает 77% результатов.**

### Модель RandomForestClassifier.

In [27]:
best_result = 0
best_model = 0

for est in range(1, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_result = result
            best_model = model

print('Accuracy лучшей модели:', best_result)
print('Лучшая модель:', best_model)

Accuracy лучшей модели: 0.8040435458786936
Лучшая модель: RandomForestClassifier(max_depth=9, n_estimators=31, random_state=12345)


**На валидационной выборке модель RandomForestClassifier предсказывает в 80% случаях.**

### Промежуточный вывод

**Исследованно две модели DecisionTreeClassifier и RandomForestClassifier.\
Модель DecisionTreeClassifier работает быстрее но предугадывает немного реже чем RandomForestClassifier.**

## Проверьте модель на тестовой выборке

### Тест модели DecisionTreeClassifier.

In [29]:
model = DecisionTreeClassifier(random_state=12345, max_depth=10)
model.fit(features_train, target_train) 
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
result

0.7962674961119751

**На тестовой выборке DecisionTreeClassifier предсказала на 2% больше правильных ответов.**

### Тест Модели RandomForestClassifier.

In [20]:
model = RandomForestClassifier(random_state=12345, n_estimators=31, max_depth=9)
model.fit(features_train, target_train) 
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
result

0.8087091757387247

**RandomForestClassifier на тестовой выборке предсказала незначительно больше правильных ответов.**

## Проверка модели на адекватность

In [42]:
strategy = ['stratified', 'most_frequent', 'uniform']

for i in strategy:
    model_dc = DummyClassifier(strategy=i, random_state=12345)
    model_dc.fit(features_train, target_train)
    model_dc_valid = model_dc.predict(features_valid)
    result = model_dc.score(model_dc_valid, target_valid)
    print('Стратегия проверки:', i)
    print( 'Accuracy Dummy:', result)
    print()

Стратегия проверки: stratified
Accuracy Dummy: 0.5847589424572317

Стратегия проверки: most_frequent
Accuracy Dummy: 0.6687402799377916

Стратегия проверки: uniform
Accuracy Dummy: 0.46967340590979784



## Общий вывод:

* **Загруженны и просмотренны данные.**
* **В данных изменен тип данных двух столбцов с float на int.**
* **Данные разбиты на три группы, train, validate, test.**

* **Исследованны две модели DecisionTreeClassifier и RandomForestClassifier, обе модели показали хороший предсказательный результат.**
* **DecisionTreeClassifier на валидационной выборке предсказал тариф в 77% случаях.**
* **RandomForestClassifier на валидационной выборке предсказал тариф в 80% случаях.**
* **Было выяснено что в случае если нужно быстродействие DecisionTreeClassifier была бы приоритетней.**
* **Если нужна более высокая точность RandomForestClassifier требует большего времени, но даёт более точный результат.**

* **Проведенны проверки моделей на тестовой выборе.**
* **DecisionTreeClassifier показал результат на 2% выше чем на валидацинной.**
* **Результат модели RandomForestClassifier на тестовой выборке незначительно выше чем на валидационной.**